In [12]:
#imports
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [13]:
#fetch url
url = "https://www.mediaite.com/daily-ratings/cable-news-ratings-wednesday-june-28-bret-baiers-whistleblower-interview-lands-him-in-second-place-overall/"
base = requests.get(url)
base

<Response [200]>

In [14]:
#convert to html text & test
base_html = base.text 
print(base_html)

<!DOCTYPE html><html lang="en"><head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5, minimum-scale=1.0">
<meta name="date" content="2023-06-29T16:41:19+00:00">
<!-- Open Graph -->
<!-- End Open Graph -->
<link rel="shortcut icon" href="https://www.mediaite.com/favicon.ico" /><link rel="apple-touch-icon" href="https://am12.mediaite.com/med/cnt/themes/m2019/images/icon-192x192.png"><link rel="icon" sizes="192x192" href="https://am12.mediaite.com/med/cnt/themes/m2019/images/icon-192x192.png">
		<!-- All in One SEO 4.4.0.1 - aioseo.com -->
		<title>Cable News Ratings Wednesday June 28: Fox News, MSNBC &amp; CNN</title>
		<meta name="description" content="Bret Baier&#039;s interview with IRS whistleblower Gary Shapley landed him the second highest-rated show on cable news for the day." />
		<meta name="robots" content="max-snippet:-1, max-image-preview:large, max-video-preview:-1" />
		<meta name="keywords" content="cable news r

In [15]:
#make soup object
soup = BeautifulSoup(base_html)
type(soup)

bs4.BeautifulSoup

In [16]:
#selecting just the first of two tables on the page
table = soup.select("table")[0]

In [17]:
#creating header list
first_row = table.select("tbody tr")[0]
header_list = [ cell.text.rstrip("\n") for cell in first_row.select("td") ]
header_list

['ET', 'FNC', 'CNN', 'MSNBC', 'NEWSMAX', 'NEWSNATION']

In [18]:
#selecting non header rows of data from table 
rows_ratings = table.select("tr")[1:19]
len(rows_ratings)
print(rows_ratings)

[<tr>
<td data-sheets-value='{"1":2,"2":"6a"}'>6a</td>
<td data-sheets-value='{"1":2,"2":"FOX AND FRIENDS:\n737"}'>FOX AND FRIENDS:<br/>
737</td>
<td data-sheets-value='{"1":2,"2":"CNN THIS MORNING:\n281"}'>CNN THIS MORNING:<br/>
281</td>
<td data-sheets-value='{"1":2,"2":"MORNING JOE:\n963"}'>MORNING JOE:<br/>
963</td>
<td data-sheets-value='{"1":2,"2":"WISE GUYS WITH JOHN TABAC:\n35"}'>WISE GUYS WITH JOHN TABAC:<br/>
35</td>
<td data-sheets-value='{"1":2,"2":"MORNING IN AMERICA:\n29"}'>MORNING IN AMERICA:<br/>
29</td>
</tr>, <tr>
<td data-sheets-value='{"1":2,"2":"7a"}'>7a</td>
<td data-sheets-value='{"1":2,"2":"FOX AND FRIENDS:\n1065"}'>FOX AND FRIENDS:<br/>
1065</td>
<td data-sheets-value='{"1":2,"2":"CNN THIS MORNING:\n310"}'>CNN THIS MORNING:<br/>
310</td>
<td data-sheets-value='{"1":2,"2":"--"}'>—</td>
<td data-sheets-value='{"1":2,"2":"WAKE UP AMERICA:\n146"}'>WAKE UP AMERICA:<br/>
146</td>
<td data-sheets-value='{"1":2,"2":"--"}'>—</td>
</tr>, <tr>
<td data-sheets-value='{"1":

In [19]:
#creating a list of cells from the table
rating_entries = []
for row in rows_ratings:
    row_cells = []
    for cell in row.select("td"):
        row_cells.append(cell.text.rstrip("\n"))
    rating_entries.append(row_cells)
rating_entries[:3]

[['6a',
  'FOX AND FRIENDS:\n737',
  'CNN THIS MORNING:\n281',
  'MORNING JOE:\n963',
  'WISE GUYS WITH JOHN TABAC:\n35',
  'MORNING IN AMERICA:\n29'],
 ['7a',
  'FOX AND FRIENDS:\n1065',
  'CNN THIS MORNING:\n310',
  '—',
  'WAKE UP AMERICA:\n146',
  '—'],
 ['8a', 'FOX AND FRIENDS:\n1199', 'CNN THIS MORNING:\n410', '—', '—', '—']]

In [20]:
cable28_df = pd.DataFrame(rating_entries, columns=header_list)
cable28_df.head()

,ET,FNC,CNN,MSNBC,NEWSMAX,NEWSNATION
0,6a,FOX AND FRIENDS:\n737,CNN THIS MORNING:\n281,MORNING JOE:\n963,WISE GUYS WITH JOHN TABAC:\n35,MORNING IN AMERICA:\n29
1,7a,FOX AND FRIENDS:\n1065,CNN THIS MORNING:\n310,—,WAKE UP AMERICA:\n146,—
2,8a,FOX AND FRIENDS:\n1199,CNN THIS MORNING:\n410,—,—,—
3,9a,AMERICAS NEWSROOM:\n1360,CNN NEWS CENTRAL:\n524,MORNING JOE:\n999,NATIONAL REPORT:\n211,—
4,10a,AMERICAS NEWSROOM:\n1335,CNN NEWS CENTRAL:\n554,ANA CABRERA REPORTS:\n868,—,NWSN LIVE W/ MARNI HUGHES:\n23


In [21]:
#dropping everything but fox to create fox column only df
fox28 = cable28_df.drop(columns=['CNN', 'MSNBC','NEWSMAX',"NEWSNATION","ET"])
fox28

,FNC
0,FOX AND FRIENDS:\n737
1,FOX AND FRIENDS:\n1065
2,FOX AND FRIENDS:\n1199
3,AMERICAS NEWSROOM:\n1360
4,AMERICAS NEWSROOM:\n1335
5,"FAULKNER FOCUS, THE:\n1251"
6,OUTNUMBERED:\n1404
7,AMERICA REPORTS:\n1055
8,AMERICA REPORTS:\n1007
9,"STORY, THE:\n1062"


In [22]:
#splitting the string and numbers in the FNC column, while dropping the :\n . And then dropping the original FNC Column
fox28[['Show', '6/28']] = fox28['FNC'].str.split(':\n', expand=True)
fox28 = fox28.drop(columns=['FNC'])
fox28

,Show,6/28
0,FOX AND FRIENDS,737
1,FOX AND FRIENDS,1065
2,FOX AND FRIENDS,1199
3,AMERICAS NEWSROOM,1360
4,AMERICAS NEWSROOM,1335
5,"FAULKNER FOCUS, THE",1251
6,OUTNUMBERED,1404
7,AMERICA REPORTS,1055
8,AMERICA REPORTS,1007
9,"STORY, THE",1062


In [23]:
fox28.to_csv("fox28.csv")